In [1]:
import os
import json
import mlflow
import subprocess

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import OpenSearchVectorSearch
from langchain.chains import RetrievalQA
from langchain_aws import BedrockLLM

from opensearchpy import RequestsHttpConnection, AWSV4SignerAuth


import sagemaker
import boto3

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/willem/.config/sagemaker/config.yaml


In [2]:
mlflow.set_tracking_uri("http://localhost:5000")

In [19]:
# Login AWS
aws_profile = "vrt-analytics-engineer-nonsensitive"
envvars = subprocess.check_output(['aws-vault', 'exec', aws_profile, '--', 'env'])
for envline in envvars.split(b'\n'):
    line = envline.decode('utf8')
    eqpos = line.find('=')
    if eqpos < 4:
        continue
    k = line[0:eqpos]
    v = line[eqpos+1:]
    if k == 'AWS_ACCESS_KEY_ID':
        aws_access_key_id = v
    if k == 'AWS_SECRET_ACCESS_KEY':
        aws_secret_access_key = v
    if k == 'AWS_SESSION_TOKEN':
        aws_session_token = v

session = boto3.Session(
aws_access_key_id, aws_secret_access_key, aws_session_token
)
credentials = session.get_credentials()
sagemaker_session = sagemaker.Session(boto_session=session)
role = sagemaker.get_execution_role(sagemaker_session=sagemaker_session)
auth = AWSV4SignerAuth(credentials, 'eu-west-2', 'aoss')

In [20]:
# 1. Get retriever
# embed stuff and write to opensearch
model_name = "NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers"
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

# Init OpenSearch client connection
docsearch = OpenSearchVectorSearch(
     index_name="aoss-index",  # TODO: use the same index-name used in the ingestion script
     embedding_function=embeddings,
     opensearch_url="https://epcavlvwitam2ivpwv4k.eu-west-2.aoss.amazonaws.com:443",  # TODO: e.g. use the AWS OpenSearch domain instantiated previously
      http_auth = auth,
      use_ssl = True,
      verify_certs = True,
      connection_class = RequestsHttpConnection,
)

In [21]:
docs = docsearch.similarity_search(
    "iets over geld en beleggen",
    k=5,
    vector_field="vrtmax_catalog_vector"
)

In [9]:
docs

[Document(metadata={'mediacontent_page_description_program': '', 'mediacontent_page_description': 'Duidingsprogramma over de economische actualiteit van de voorbije week.', 'mediacontent_page_editorialtitle_program': 'Duiding bij de economische actualiteit', 'mediacontent_pagetitle_program': 'De markt', 'mediacontent_pagetitle_season': 'Seizoen 2023', 'mediacontent_pagetitle': 'De markt 20230915', 'offering_publication_planneduntil': '2024-09-15 21:55:00.000', 'brand_contentbrand': 'vrtnws', 'mediacontent_pageurl': 'https://www.vrt.be/vrtmax/a-z/de-markt/2023/de-markt-d20230915/', 'mediacontent_imageurl': 'https://images.vrt.be/orig/2023/09/02/297a9680-4913-11ee-91d7-02b7b76bf47f.jpg', 'mediacontent_programimageurl': '//images.vrt.be/orig/2019/10/04/176cfbcb-e6b8-11e9-abcc-02b7b76bf47f.jpg'}, page_content="Obligaties die een rente van 6% beloven. Is dat, na jaren van lage rente, de manier om spaargeld te activeren? Waarom zijn er meer vrouwen in een mannenbastion als de ITsector nodig?

In [5]:
def loader_fn():
    model_name = "NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers"
    encode_kwargs = {'normalize_embeddings': False}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        encode_kwargs=encode_kwargs
    )
    # Init OpenSearch client connection
    docsearch = OpenSearchVectorSearch(
        index_name="aoss-index",  # TODO: use the same index-name used in the ingestion script
        embedding_function=embeddings,
        opensearch_url="https://epcavlvwitam2ivpwv4k.eu-west-2.aoss.amazonaws.com:443",  # TODO: e.g. use the AWS OpenSearch domain instantiated previously
    #     http_auth=("<<insert_user_name>>", "<<insert_password>>"),
    #     use_ssl = False,
    #     verify_certs = False,
    #     ssl_assert_hostname = False,
    #     ssl_show_warn = False,
    )

    return docsearch.as_retriever()

In [6]:
# 2. Get LLM
llm = BedrockLLM(
    credentials_profile_name="vrt-analytics-engineer-nonsensitive", 
    region_name="eu-west-2",
    model_id="meta.llama3-70b-instruct-v1:0"
)

In [7]:
# 3. Create model in MLFLOW
mlflow.set_experiment("/langchain_rag")

retrievalQA = RetrievalQA.from_llm(llm=llm, retriever=docsearch.as_retriever())

with mlflow.start_run() as run:
    model_info = mlflow.langchain.log_model(
        retrievalQA,
        artifact_path="rag",
        loader_fn=loader_fn
    )

2024/08/06 23:56:54 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp69ka3hx1/model, flavor: langchain). Fall back to return ['langchain==0.2.8', 'pydantic==2.8.2', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 
/home/willem/miniconda3/envs/llm/lib/python3.10/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [8]:
model_info.predict([{"input": "iets over geld en belegggen"}])


AttributeError: 'ModelInfo' object has no attribute 'predict'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
retriever = docsearch.as_retriever()

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

chain.invoke({"input": "iets over geld en beleggen"})

AuthorizationException: AuthorizationException(403, 'Forbidden')